In [1]:
import pandas as pd
import datatable as dt
import dask.dataframe as dd
import numpy as np 
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras 
import keras.backend as K
from tensorflow.keras.callbacks import  EarlyStopping
from tensorflow.keras.layers import Input, Dense, Lambda
from tensorflow.keras.layers import Flatten, concatenate
from tensorflow.keras.layers import BatchNormalization, Dropout, Reshape
from keras.utils.vis_utils import plot_model
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler

In [2]:
df = dt.fread(r"\\wsl.localhost\Ubuntu-22.04\home\imperious\ML and CVAE projects\GW from EMRI\gw data\GW_data\results\datasignal_one_1.txt",sep="\t")
df[dt.str32] = dt.float64
df

,C0,C1,C2,C3,C4,C5,C6,C7,C8,C9,…,C1577903,C1577904,C1577905,C1577906,C1577907
,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪
0,3.83985e-22,7.69554e-22,1.13743e-21,1.44821e-21,1.6863e-21,1.83974e-21,1.90084e-21,1.86652e-21,1.73852e-21,1.52325e-21,…,4.45936e-22,−2.50303e-21,−3.98991e-21,−4.32521e-21,−4.13826e-21
1,−1.01416e-21,−7.74709e-22,−4.99028e-22,−2.07651e-22,9.02605e-23,3.85339e-22,6.68307e-22,9.30265e-22,1.16298e-21,1.35912e-21,…,−4.13849e-21,−3.49586e-21,−2.28605e-21,−2.24409e-21,−3.91962e-21
2,1.20966e-21,1.13838e-21,1.04019e-21,9.20266e-22,7.81108e-22,6.25626e-22,4.5707e-22,2.7896e-22,9.50192e-23,−9.09095e-23,…,5.23381e-23,−3.91688e-22,−8.04927e-22,−1.16927e-21,−1.47168e-21
3,−2.42755e-21,−2.25382e-21,−2.06635e-21,−1.87457e-21,−1.67887e-21,−1.47966e-21,−1.27737e-21,−1.0724e-21,−8.65188e-22,−6.5617e-22,…,−1.21095e-21,−1.4834e-21,−1.75128e-21,−2.01394e-21,−2.27078e-21
4,−6.66703e-22,−6.96611e-22,−7.26391e-22,−7.5435e-22,−7.80419e-22,−8.04532e-22,−8.26629e-22,−8.46655e-22,−8.6456e-22,−8.80298e-22,…,−1.01421e-21,−1.02737e-21,−1.03796e-21,−1.04596e-21,−1.05137e-21
5,2.85836e-22,1.08388e-22,−1.08196e-22,−2.9329e-22,−3.93027e-22,−3.78379e-22,−2.53611e-22,−5.50333e-23,1.5956e-22,3.27715e-22,…,−6.98691e-22,1.11654e-21,1.07112e-21,9.94455e-22,1.08191e-21
6,−1.23525e-21,−1.05868e-21,−8.7101e-22,−6.81826e-22,−4.91454e-22,−3.00223e-22,−1.08468e-22,8.34783e-23,2.75282e-22,4.66609e-22,…,−5.17145e-21,−5.07792e-21,−4.97446e-21,−4.86139e-21,−4.73902e-21
7,2.66315e-21,2.78778e-21,2.91335e-21,3.0329e-21,3.14618e-21,3.25295e-21,3.35299e-21,3.4461e-21,3.53209e-21,3.61077e-21,…,8.35826e-23,2.85423e-22,4.86438e-22,6.86225e-22,8.84389e-22
8,8.56012e-21,8.36924e-21,8.14821e-21,7.90709e-21,7.64647e-21,7.36699e-21,7.06935e-21,6.75429e-21,6.42256e-21,6.07501e-21,…,7.88476e-21,8.36311e-21,8.8129e-21,9.2331e-21,9.6228e-21


In [3]:
params = dt.fread(r"\\wsl.localhost\Ubuntu-22.04\home\imperious\ML and CVAE projects\GW from EMRI\gw data\readme\paramlist.txt",sep="\t")
params=params[:250,:]
params

,M,mu,a,r0,phi0,thetas,phis,theta1,phi1
,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪
0,700000,40,0,13.8743,0,(57*Pi)/100,(19*Pi)/100,Pi/25,Pi
1,800000,72,0,14.8416,1.94445,(37*Pi)/100,(4*Pi)/25,(21*Pi)/50,(3*Pi)/4
2,2000000,34,0,9.23585,0.00356876,(49*Pi)/100,(43*Pi)/50,(37*Pi)/100,(31*Pi)/100
3,9000000,86,0,7.28645,2.04012,(3*Pi)/4,(12*Pi)/25,(18*Pi)/25,(99*Pi)/100
4,9000000,34,0,6.83257,−0.747715,(73*Pi)/100,(47*Pi)/100,(19*Pi)/50,(47*Pi)/50
5,200000,10,0,17.6797,−1.81508,(23*Pi)/100,(43*Pi)/50,(31*Pi)/100,(3*Pi)/100
6,10000000,96,0,7.2278,1.44778,Pi/3,Pi/4,Pi/8,0
7,10000000,40,0,6.81373,1.03285,(21*Pi)/25,(51*Pi)/100,(99*Pi)/100,(3*Pi)/50
8,8000000,100,0,7.53638,−2.00043,(2*Pi)/25,(23*Pi)/100,(87*Pi)/100,(37*Pi)/100


In [4]:
continuous_X = params[:,:5]
normalizer_X = MinMaxScaler()
normalizer_X.fit(continuous_X) #Fitted with continuous_x. Use it to decode MinMax.

cont_norm_X = normalizer_X.transform(continuous_X)

#print(pd.DataFrame(cont_norm_X))

categorical_X = (dt.Frame.to_pandas(params[:,5:]))
encoderOHE = OneHotEncoder()
encoderOHE.fit(categorical_X) #Fitted with categorical_x. Use it to decode OneHot.

cat_ohe_X = encoderOHE.transform(categorical_X)

#print(pd.DataFrame(cat_ohe_X.toarray()))

#normalizing gw data
normalizer_cond = MinMaxScaler()
normalizer_cond.fit(df)

df = normalizer_cond.transform(df) #does sending to the df itself save ram?



In [5]:
print(pd.DataFrame(cat_ohe_X.toarray()))

     0    1    2    3    4    5    6    7    8    9    ...  332  333  334  \
0    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
1    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
2    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
3    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  1.0   
4    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
..   ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
245  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
246  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
247  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
248  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
249  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   

     335  336  337  338  339  340  341  
0    0.0  1.0  0.0  0.0  0.0  0.0 

In [6]:
hidden_dim = 2
batch_size = 25
num_classes = 10 

In [7]:
def dropout_and_batchnorm(x):
    return Dropout(0.3)(BatchNormalization()(x))

def noiser(args):
    global mean, log_var
    mean, log_var = args
    N = K.random_normal(shape=(batch_size, hidden_dim), mean=0., stddev=1.0)
    return K.exp(log_var / 2) * N + mean

In [8]:
# Encoder




input_params_cont = Input(shape=(5,))
input_params_cat = Input(shape=(342,))
cond_gw = Input(shape=(1577907,))

x = concatenate([input_params_cont, input_params_cat, cond_gw])
x = Dense(256, activation="relu")(x)
x = dropout_and_batchnorm(x)
x = Dense(128, activation="relu")(x)
x = dropout_and_batchnorm(x)

In [9]:
# Latent Space
mean = Dense(hidden_dim)(x)
log_var = Dense(hidden_dim)(x)
h = Lambda(noiser, output_shape=(hidden_dim,), name="latent_space")([mean, log_var])

In [10]:
# Decoder
input_decoder = Input(shape=(hidden_dim,))
cond_gw_decoder = Input(shape=(1577907,))
d = concatenate([input_decoder,cond_gw_decoder])
d = Dense(128, activation="relu")(d)
d = dropout_and_batchnorm(d)
d = Dense(256, activation="relu")(d)
d = dropout_and_batchnorm(d)
d1 = Dense(342, activation="sigmoid")(d)
d2 = Dense(5, activation="linear")(d)



In [11]:
def vae_loss1(x, y):
    x = K.reshape(x, shape=(342,)) #Corrigir
    y = K.reshape(y, shape=(342,)) #Corrigir
    loss = K.sum(K.square(x-y), axis=-1)
    kl_loss = -0.5 * K.sum(1 + log_var - K.square(mean) - K.exp(log_var), axis=-1)
    return loss + kl_loss

def vae_loss2(x, y):
    x = K.reshape(x, shape=(5,)) #Corrigir
    y = K.reshape(y, shape=(5,)) #Corrigir
    loss = K.sum(K.square(x-y), axis=-1)
    kl_loss = -0.5 * K.sum(1 + log_var - K.square(mean) - K.exp(log_var), axis=-1)
    return loss + kl_loss

In [ ]:
encoder = keras.Model([input_params_cat,input_params_cont,cond_gw], h, name="encoder") # Parece estar certo
decoder = keras.Model([input_decoder, cond_gw_decoder], [d1,d2], name="decoder") #corrigir

cvae = keras.Model(inputs = [input_params_cat,input_params_cont, cond_gw, cond_gw_decoder], #observar se está certo
                   outputs = decoder([encoder([input_params_cat,input_params_cont, cond_gw]), cond_gw_decoder]),
                   name="cvae")

cvae.compile(optimizer="adam", loss={})

plot_model(encoder, to_file='encoder_plot.png', show_shapes=True, show_layer_names=True)
plot_model(decoder, to_file='decoder_plot.png', show_shapes=True, show_layer_names=True)

cvae.summary()

In [ ]:
epochs = 25

es = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=10)

cvae.fit(
    [x_train, y_train_cat, y_train_cat], x_train,
    epochs=epochs,
    batch_size=batch_size,
    shuffle=True
)